In [1]:
#libraries imported

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle
from tqdm.notebook import tqdm

In [2]:
spotify_songs = pd.read_csv(r'C:\Users\temsy\Documents\GitHub\Spotifinder\spotify_songs.csv')

In [3]:
#Drop null values in lyrics column

spotify_songs =  spotify_songs.dropna()

In [4]:
spotify_songs.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [5]:
spotify_songs['language'].value_counts()

en    15405
es     1705
de      241
pt      176
it      133
pl       76
nl       72
fr       65
tl       63
da       40
id       34
ko       29
vi       24
so       16
ro       16
sw       16
cy       12
hi       10
no       10
ja        8
et        7
tr        5
af        5
sv        5
ru        4
hr        3
sq        3
ar        2
ca        2
el        2
cs        2
fi        1
hu        1
sk        1
Name: language, dtype: int64

In [6]:
#Only get songs with lyrics in English

spotify_songs = spotify_songs[spotify_songs['language']=='en']

In [7]:
spotify_songs['track_popularity'].value_counts()

0      1355
1       327
51      296
60      286
57      285
       ... 
93        4
97        3
95        2
99        1
100       1
Name: track_popularity, Length: 100, dtype: int64

In [8]:
#Only get the most popular 5000 songs

df1 = spotify_songs.sort_values('track_popularity', ascending = False)
df1 = df1[0:5000]
df1.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
6925,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,Tones and I,"They say, ""Oh my God, I see the way you shine ...",100,0UywfDKYlyiu1b38DRrzYD,Dance Monkey (Stripped Back) / Dance Monkey,2019-10-17,"post-teen alternative, indie, pop (large variety)",1y42gwI5cuwjBslPyQNfqb,...,-6.400,0,0.0924,0.69200,0.000104,0.1490,0.513,98.027,209438,en
14477,696DnlkuDOXcMAnKlTgXXK,ROXANNE,Arizona Zervas,"All for the 'Gram Bitches love the 'Gram Oh, w...",99,6HJDrXs0hpebaRFKA1sF90,ROXANNE,2019-10-10,Global Top 50 | 2020 Hits,1KNl4AYfgZtOVm9KHkhPTF,...,-5.616,0,0.1480,0.05220,0.000000,0.4600,0.457,116.735,163636,en
4813,21jGcNKet2qwijlDFuPiPb,Circles,Post Malone,"Oh, oh, oh Oh, oh, oh Oh, oh, oh, oh, oh We ...",98,4g1ZRSobMefqF6nelkgibi,Hollywood's Bleeding,2019-09-06,Pop - Pop UK - 2019 - Canadian Pop - 2019 - Pop,46Cl6dmeiylK6TRGXr7hHe,...,-3.497,1,0.0395,0.19200,0.002440,0.0863,0.553,120.042,215280,en
1754,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,Yeah I've been tryna call I've been on my own ...,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,2019-11-29,Todo Éxitos,2ji5tRQVfnhaX1w9FhmSzk,...,-4.075,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,en
1375,0nbXyq5TXYPCO7pr3N8S4I,The Box,Roddy Ricch,Pullin' out the coupe at the lot Told 'em fuck...,98,52u4anZbHd6UInnmHRFzba,Please Excuse Me For Being Antisocial,2019-12-06,RapCaviar,37i9dQZF1DX0XUsuxWHRQd,...,-6.687,0,0.0559,0.10400,0.000000,0.7900,0.642,116.971,196653,en


In [9]:
#Create a function that gathers all the song lyrics into a list of strings

def gather_data(songs):
  data =[]
  for song in songs:
    data.append(spotify_songs['lyrics'][spotify_songs['track_id']==song])
  return data

In [10]:
songs = df1['track_id']

data = gather_data(songs)

In [11]:
#Convert Series into a list of strings

new_data = []

for song in data:
  str_song = pd.Series(song).item()
  new_data.append(str_song)

In [13]:
#Apply TD-IDF to the corpus

#Instantiate
tfidf = TfidfVectorizer(stop_words='english',
                        ngram_range=(1,2),
                        min_df=3,
                        max_df=0.25)

#Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(new_data)

#Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
print(dtm.shape)
dtm.head()

(5000, 39317)


,00,000,000 miles,01,03,06,10,10 10,10 30,10 got,...,하지,한마디면,한마디면 like,한번,해도,헤매어,헤벌레,헤벌레 따뜻한,흐르게,흐르게 된다면
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#Apply K Nearest Neighbors

#Fit on DTM
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree')

In [15]:
dtm.head()

,00,000,000 miles,01,03,06,10,10 10,10 30,10 got,...,하지,한마디면,한마디면 like,한번,해도,헤매어,헤벌레,헤벌레 따뜻한,흐르게,흐르게 된다면
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
dtm.shape

(5000, 39317)

In [17]:
new_data[0]

'They say, "Oh my God, I see the way you shine Take your hands, my dear, and place them both in mine" You know you stopped me dead while I was passing by And now I beg to see you dance just one more time Ooh, I see you, see you, see you every time And oh my, I, I like your style You, you make me, make me, make me wanna cry And now I beg to see you dance just one more time So they say Dance for me, dance for me, dance for me, oh-oh-oh I\'ve never seen anybody do the things you do before They say Move for me, move for me, move for me, ayy-ayy-ayy And when you\'re done, I\'ll make you do it all again I said, "Oh my God, I see you walking by Take my hands, my dear, and look me in my eyes"Just like a monkey, I\'ve been dancin\' my whole life But you just beg to see me dance just one more time Ooh, I see you, see you, see you every time And oh my, I, I like your style You, you make me, make me, make me wanna cry And now I beg to see you dance just one more time So they say Dance for me, danc

In [19]:
#Query a document
x = 0 
doc = dtm.iloc[x].values

#Query using kneighbors
nn.kneighbors([doc])

(array([[0.        , 0.00551693, 0.00827337, 1.        , 1.08085923]]),
 array([[   0,  424,   20, 4617, 1610]], dtype=int64))

In [23]:
new_data[4617]

'(One, two, three)'

In [24]:
new_data[1610]

"In the faded light you touch my body I can feel your hands on my skin Think you got me right where you want me But you're just in my way I came to party on my own Don't need nobody in my zone I get down to the beat, I lose control Eh oh, I go so so-lo Boy you can cool it down Not here to fool around Just wanna dance, dance, dance Dance, dance, dance I know you want me But I don't care, baby Just want to dance, dance, dance Dance, dance, dance (Dance, dance, dance) Dance, dance, dance (Dance, dance, dance) In the shade of night You're moving closer We're on our way to cross a line Think you got me right where you want me But it's all in your mind I came to party on my own Don't need nobody in my zone I get down to the beat, I lose control Eh oh, I go so so-lo Boy you can cool it down Not here to fool around Just wanna dance, dance, dance Dance, dance, dance I know you want me But I don't care, baby Just want to dance, dance, dance Dance, dance, dance (Dance, dance, dance) Dance, dance,

In [32]:
df1.iloc[424]

track_id                                               1rgnBhdG2JDFTbYkYRZAku
track_name                                                       Dance Monkey
track_artist                                                      Tones and I
lyrics                      They say, "Oh my God, I see the way you shine ...
track_popularity                                                           79
track_album_id                                         31IDBea3eEs57a0joX6TjN
track_album_name                                                 Dance Monkey
track_album_release_date                                           2019-05-10
playlist_name                 Pop - Pop UK - 2019 - Canadian Pop - 2019 - Pop
playlist_id                                            46Cl6dmeiylK6TRGXr7hHe
playlist_genre                                                            pop
playlist_subgenre                                               post-teen pop
danceability                                                    

In [ ]:
#Serialize model 

nnPickle = open('nnpickle_file', 'wb')

#source, destination
pickle.dump(nn, nnPickle)

#load the model from disk
loaded_model = pickle.load(open('nnpickle_file', 'rb'))
result = loaded_model.predict(song)

In [40]:
#Serialize model

import pickle

#save the model to disk

filename = 'nlp_model.sav'

pickle.dump(nn, open(filename, 'wb'))

In [41]:
#Load and test the model

x = 0 #how to figure out which row is the selected song
doc = dtm.iloc[x].values

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.kneighbors([doc])
print(result)

(array([[0.        , 0.00551693, 0.00827337, 1.        , 1.08085923]]), array([[   0,  424,   20, 4617, 1610]], dtype=int64))


In [52]:
result[1][0][4]

1610

In [114]:
#function to get 4 nearest songs 

def select_nearest_songs(artist, song):
    loaded_model = pickle.load(open('nlp_model.sav', 'rb'))
    
    #translate artist, song into doc dtm.iloc[x].values
    artist_songs = df1.loc[df1['track_artist']==artist]
    selected_song = artist_songs.loc[artist_songs['track_name']==song]
    x = selected_song.index
    x = x.item()

    doc = dtm.loc[x].values
    result = loaded_model.kneighbors([doc])
    
    song1 = result[1][0][1] #gives the loc

    #translate the loc into an artist and song title
    artist1 = spotify_songs.loc[song1]['track_artist']
    song1 = spotify_songs.loc[song1]['track_name']

    #translate result into song names
    return artist1, song1

In [115]:
select_nearest_songs('Post Malone', 'Circles')

('Deitrick Haddon', 'Power')